In [1]:
import pandas as pd
import numpy as np
import pickle
import os

In [ ]:
file_path = "./lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv"

In [ ]:
# df = pd.read_csv(
#     file_path, sep='\t', header=None,
#     names=[
#         'user_id', 'timestamp', 'artist_id', 'artist_name', 'track_id', 'track_name'
#     ],
#     skiprows=[
#         2120260-1, 2446318-1, 11141081-1,
#         11152099-1, 11152402-1, 11882087-1,
#         12902539-1, 12935044-1, 17589539-1
#     ]
# )
# df["timestamp"] = pd.to_datetime(df.timestamp)
# df.sort_values(['user_id', 'timestamp'], ascending=True, inplace=True)
# print(f'Number of Records: {len(df):,}\nUnique Users: {df.user_id.nunique()}\nUnique Artist:{df.artist_id.nunique():,}')
# df.head(5)

In [ ]:
# save_filepath = os.path.join("./lastfm-dataset-1K", 'lastfm-dataset-1k.snappy.parquet')

# df.to_parquet(save_filepath, compression='GZIP', index=False)

In [2]:
save_filepath = os.path.join("./lastfm-dataset-1K", 'lastfm-dataset-1k.snappy.parquet')

df = pd.read_parquet(save_filepath)
print(f'Number of Records: {len(df):,}\nUnique Users: {df.user_id.nunique()}\nUnique Artist:{df.artist_id.nunique():,}')
df.head(5)

Number of Records: 19,098,853
Unique Users: 992
Unique Artist:107,295
CPU times: user 34.1 s, sys: 10.4 s, total: 44.4 s
Wall time: 44.7 s


,user_id,timestamp,artist_id,artist_name,track_id,track_name
0,user_000001,2006-08-13 13:59:20+00:00,09a114d9-7723-4e14-b524-379697f6d2b5,Plaid & Bob Jaroc,c4633ab1-e715-477f-8685-afa5f2058e42,The Launching Of Big Face
1,user_000001,2006-08-13 14:03:29+00:00,09a114d9-7723-4e14-b524-379697f6d2b5,Plaid & Bob Jaroc,bc2765af-208c-44c5-b3b0-cf597a646660,Zn Zero
2,user_000001,2006-08-13 14:10:43+00:00,09a114d9-7723-4e14-b524-379697f6d2b5,Plaid & Bob Jaroc,aa9c5a80-5cbe-42aa-a966-eb3cfa37d832,The Return Of Super Barrio - End Credits
3,user_000001,2006-08-13 14:17:40+00:00,67fb65b5-6589-47f0-9371-8a40eb268dfb,Tommy Guerrero,d9b1c1da-7e47-4f97-a135-77260f2f559d,Mission Flats
4,user_000001,2006-08-13 14:19:06+00:00,1cfbc7d1-299c-46e6-ba4c-1facb84ba435,Artful Dodger,120bb01c-03e4-465f-94a0-dce5e9fac711,What You Gonna Do?


In [ ]:
# sub_dataset = df[df.track_id.isin(rand_track)]

In [ ]:
# sub_dataset = sub_dataset.reset_index(drop=True)

In [ ]:
df = df.fillna("None")

tot_track = df.track_id.nunique()

rand_track = df.track_id.unique()[np.random.choice(range(tot_track), 199001, replace=False)]

df = df[df.track_id.isin(rand_track)]

In [ ]:
tot_session_num = len(df)

In [ ]:
# session_id = np.array(range(tot_session_num)) // 20

In [ ]:
# sub_dataset.insert(loc=0, column="session_id", value=session_id)

In [ ]:
# sub_dataset.insert(loc=0, column="keep", value=session_id)

In [ ]:
# df.groupby("user_id").count().describe()

In [ ]:
# sub_dataset.groupby('session_id').apply(lambda x: x.timestamp.max() - x.timestamp.min() < pd.Timedelta("02:00:00"))

In [ ]:
seperator = np.array(range(tot_session_num)) // 20

In [ ]:
# sub_dataset.columns.insert(0, "sep")

In [ ]:
last = 0
def process(ee: pd.DataFrame):
    global last
    # ee = ee.sort_values("timestamp")

    ee.insert(0, "sep", seperator[:len(ee)] + last + 1)

    last = seperator[len(ee) - 1] + last + 1
    return ee

def process2(ee: pd.DataFrame):
    ee.insert(0, "keep", len(ee) >= 5 and ee.timestamp.max() - ee.timestamp.min() < pd.Timedelta("02:00:00"))

    return ee

In [ ]:
split_dataset = df.groupby('user_id').apply(lambda x: process(x))

In [ ]:
kept_dataset = split_dataset.groupby("sep").apply(lambda x: process2(x))

In [ ]:
kept_dataset

In [ ]:
final_dataset = kept_dataset[kept_dataset.keep]

In [ ]:
final_dataset = final_dataset.reset_index(drop=True).drop(columns=["keep"])

In [ ]:
def reindex(df: pd.DataFrame, columns:list):
    for column in columns:
        df[column] = df[column].map(dict(zip(df[column].unique(), range(1, len(df[column].unique())+1))))

In [ ]:
# reindex(final_dataset, ["sep"])

In [ ]:
final_dataset.to_csv("./lastfm-dataset-1K/filtered_session_199000_tracks.csv", index=False)

In [ ]:
cp = final_dataset.copy()
map_user = dict(zip(cp["user_id"].unique(), range(1, len(cp["user_id"].unique())+1)))
map_track = dict(zip(cp["track_id"].unique(), range(1, len(cp["track_id"].unique())+1)))

pickle.dump([map_user, map_track], open("./lastfm-dataset-1K/user_track_id_map_199000.pkl", "wb"))

In [ ]:
reindex(cp, ["sep", "user_id", "track_id"])

In [ ]:
cp.groupby('sep').count().describe()

In [ ]:
cp.to_csv("reindexed_sessions.csv", index=False)